In [2]:
# Specify the requirements to a text file
%%writefile requirements.txt
greenlet==0.4.13
pytorch-pretrained-bert==0.6.1
allennlp==0.8.2

Writing requirements.txt


In [3]:
# Install dependencies
!pip install -r requirements.txt

     |████████████████████████████████| 51kB 2.7MB/s 
     |████████████████████████████████| 122kB 10.3MB/s 
     |████████████████████████████████| 5.6MB 14.4MB/s 
     |████████████████████████████████| 71kB 11.7MB/s 
     |████████████████████████████████| 133kB 54.2MB/s 
     |████████████████████████████████| 4.5MB 55.5MB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
     |████████████████████████████████| 317kB 59.9MB/s 
     |████████████████████████████████| 245kB 48.6MB/s 
     |████████████████████████████████| 512kB 53.0MB/s 
     |████████████████████████████████| 1.3MB 54.4MB/s 
     |████████████████████████████████| 552kB 48.3MB/s 
     |████████████████████████████████| 92kB 10.8MB/s 
     |████████████████████████████████| 25.2MB 120kB/s 
     |████████████████████████████████| 12.6MB 249kB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 2.7MB 35.3MB/s 
     |████████████████████████████████| 61kB 9.7MB/s 
  

In [4]:
!pip install scikit-learn==0.22.2
from allennlp.predictors.predictor import Predictor

# Specify path for Bi-directional Attention Flow model to be used for prediction
predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/bidaf-model-2017.09.15-charpad.tar.gz")

     |████████████████████████████████| 7.1MB 4.7MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
100%|██████████| 46175392/46175392 [00:00<00:00, 53092120.48B/s]
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/usr/local/lib/python3.6/dist-packages/allennlp/data/token_indexers/token_characters_indexer.py:51: UserWarning: You are using the default value (0) of `min_padding_length`, which can cause some subtle bugs (more info see https://github.com/allenai/allennlp/issues/1954). Strongly recommend to set a value, usually the maximum size of the convolutional layer size when using CnnEncoder.
  UserWarning)


In [5]:
# Text extraction
from bs4 import BeautifulSoup # Use BeautifulSoup library for web scraping
from bs4.element import Comment
import urllib.request

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

In [6]:
# Ignore warnings in output cells
import warnings
warnings.simplefilter("ignore", UserWarning)

In [7]:
# Query the relevant webpage to extract text from (Wikipedia in this case)
html = urllib.request.urlopen('https://en.wikipedia.org/wiki/David_Blaine').read()
question = input()
result=predictor.predict(
  passage=text_from_html(html),
  question=question)
print('\033[1m\nANSWER\n\n' + result['best_span_str'] + '\n') # \033[1m is used to print bold output

How long did David Blaine hold his breath underwater?

ANSWER

17 minutes



In [8]:
# Using my github repo for this example
html = urllib.request.urlopen('https://github.com/ushashwat/Hybrid-Movie-Recommender').read()
question = input()
result=predictor.predict(
  passage=text_from_html(html),
  question=question)
print('\033[1m\nANSWER\n\n' + result['best_span_str'] + '\n')

Which dataset was used here?

ANSWER

Full MovieLens Dataset



In [9]:
# Using the text from TMDb API page for Q&A
html = urllib.request.urlopen('https://www.themoviedb.org/documentation/api').read()
question = input()
result=predictor.predict(
  passage=text_from_html(html),
  question=question)
print('\033[1m\nANSWER\n\n' + result['best_span_str'] + '\n')

Does TMDB api cost anything?

ANSWER

free to use as long as you attribute TMDb as the source of the data and/or images



In [10]:
# Using my friend's blog as the final example
html = urllib.request.urlopen('https://mellowedmusings.home.blog/2019/09/18/climate-change/').read()
question = input()
result=predictor.predict(
  passage=text_from_html(html),
  question=question)
print('\033[1m\nANSWER\n\n' + result['best_span_str'] + '\n')

What are the pillars of sustainability?

ANSWER

economy, ecology, and society

